In [1]:
from instrumentdatabaseapi import instrumentdatabaseapi as API

repo = API.Repository(local_repo=".")

In [ ]:
print(repo)


Load the instrument configuration providing:
 - name of the institute
 - name of the instrument
 - version of the instrument description (HEAD is the most recent)
 - name of the simulation program

In [ ]:
#help(repo)

In [ ]:
repo.ls_institutes()

In [2]:
repo.ls_instruments("ILL")
instrument_name="Panther"
instrument_name="ThALES"

Available instruments for ILL:
 -  ILL/Panther
 -  ILL/D22
 -  ILL/ThALES


In [ ]:
repo.ls_versions("ILL",instrument_name)

In [ ]:
repo.ls_simulation_programs("ILL",instrument_name,"HEAD")

In [ ]:
repo.ls_flavours("ILL",instrument_name,"HEAD","mcstas")

In [3]:
flavour="full"
flavour="nosection"

myinstrument = repo.load("ILL", instrument_name, "HEAD", "mcstas", flavour, dep=False)

In [ ]:
myinstrument.list_calculators()

In [ ]:
print(myinstrument)

In [4]:
# setting the base directory for the simulation output
basedir = "/tmp/"+instrument_name+"/"
myinstrument.set_instrument_base_dir(basedir)
import os 
os.mkdir(basedir)


FileExistsError: [Errno 17] File exists: '/tmp/ThALES/'

In [ ]:
myinstrument.master

In [5]:
import pint
ureg = pint.get_application_registry()

if instrument_name == "ThALES":
    myinstrument.master["a2"] = myinstrument.energy_to_angle(4.98 * ureg.meV)
    myinstrument.master["a4"] = 60 * ureg.degree
    myinstrument.master["a6"] = myinstrument.master["a2"].pint_value
if instrument_name == "Panther":
    myinstrument.master["energy"]=19 * ureg.meV

In [ ]:
myinstrument.samples

In [6]:
myinstrument.set_sample_by_name("vanadium")

SPLIT 104 COMPONENT sqw = Isotropic_Sqw(
  Sqw_coh = 0, // [str]
  Sqw_inc = 0, // [str]
  radius = sample_radius, // [m]
  thickness = sample_thickness, // [m]
  xwidth = sample_width, // [m]
  yheight = sample_height, // [m]
  verbose = 2, // [1]
  d_phi = 9.147842519801722, // [deg]
  rho = 0.07232226802632531, // [AA-3]
  sigma_abs = 5.08, // [barns]
  sigma_coh = 0, // [barns]
  sigma_inc = 4.935, // [barns]
  p_interact = 1 // [1]
)
AT (0, 0, 0) RELATIVE sample_arm

In [7]:
myinstrument.sample_cylinder_shape(0.005, 0.01)

In [8]:
myinstrument.sim_neutrons(5e6)
myinstrument.set_seed(654321)

### Diagnostics of the beam for one calculator

In [ ]:
calculator_name="SampleCalc"
calculator_name="OriginCalc"
mycalc = myinstrument.calculators[calculator_name]
mycalc.backengine()
#myinstrument.run()
print(mycalc.output)

In [ ]:
#mycalc = myinstrument.calculators["OriginCalc"]
#mycalc = myinstrument.calculators["SampleCalc"]
#mycalc.input = myinstrument.calculators["OriginCalc"].output
mycalc.show_diagram()


In [ ]:
mycalc.show_diagram(analysis=True)

In [ ]:
import mcstasscript as ms


In [ ]:
diag = ms.Diagnostics(mycalc)
diag.settings(ncount=1e7, suppress_output=True)
diag.show_settings()
diag.clear_points()

elements = []
if len(elements) == 0:
    for component in diag.instr.component_list:
        if component.component_name in ["Progress_bar", "Arm"]:
            continue
        if component.component_name in ["Beamstop"]:
            diag.add_point(before=component.name)
            continue
        elements.append(component.name)
        diag.add_point(after=component.name)
        

print(elements)


if mycalc.name == "SampleCalc":
    diag.clear_points()
    diag.add_point(after="Vin")
    diag.add_point(before="sqw")
    diag.add_point(after="Sample_Out")
    diag.add_point(before="after_sample_slit")
    diag.add_point(after="after_sample_slit")

print(diag)


In [ ]:
diag.run()

In [ ]:
diag.clear_views()
#diag.add_view("t")
diag.add_view("x", bins=50, limits=[-0.04,0.04])
diag.add_view("y", bins=50, same_scale=False, limits=[-0.07,0.07])
diag.add_view("x","y",bins=[30,30])
diag.add_view("e", same_scale=True)
diag.add_view("t",same_scale=False)
diag.add_view("dx","dy")
print(diag)

In [ ]:
diag.plot()

In [ ]:
mycalc.show_instrument()

In [ ]:
ms.make_sub_plot(data_empty[55:59])

In [ ]:
ms.make_sub_plot(data_empty[65:69])

In [ ]:
%matplotlib widget
import mcstasscript.jb_interface as ms_widget
ms_widget.show(data_empty)

In [ ]:
myinstrument.sample.set_SPLIT(60)
myinstrument.sample

In [ ]:
slit=myinstrument.calculators["SampleCalc"].get_component("after_sample_slit")
slit.EXTEND=""
slit.append_EXTEND("if(p<1e-1) ABSORB;")
slit


In [ ]:
myinstrument.run()

In [ ]:
data=myinstrument.output
print(data)
calcname_data="OriginCalc_data"

In [ ]:
print(data[calcname_data])
for detector in data[calcname_data].get_data()["data"]:
    print(detector.Intensity, detector.Error, detector.Ncount)

In [ ]:
(data[calcname_data]).get_data()["data"][0].Intensity

In [ ]:
wmins=["1e-5", "1e-4", "1e-3", "1e-2", "1e-1"]
detectors=[]
for w_min in wmins:
    slit=myinstrument.calculators["SampleCalc"].get_component("after_sample_slit")
    slit.EXTEND=""
    slit.append_EXTEND(f"if(p<{w_min}) ABSORB;")
    myinstrument.run()
    data=myinstrument.output
    detector=(data["DetectorCalc_data"].get_data()["data"][0])
    detectors.append(detector)

In [ ]:
detectors

In [9]:
import numpy as np
diags = []
RHs = list(np.arange(2.6,4,0.3))
RVs = list(np.arange(1,3,0.3))
RVs.append(1.88)
RHs.append(3.31)
print(RHs)
print(RVs)

[2.6, 2.9, 3.1999999999999997, 3.4999999999999996, 3.7999999999999994, 3.31]
[1.0, 1.3, 1.6, 1.9000000000000001, 2.2, 2.5, 2.8000000000000003, 1.88]


In [ ]:
for RH in RHs:
    print(RH)
    mycalc.parameters["RHmono"]=RH
    diag = ms.Diagnostics(mycalc)
    diag.settings(ncount=1e6, suppress_output=True,force_compile=False)
    diag.clear_points()
    diag.add_point(after="before_sample_slit")
#    diag.add_point(before="sqw")
    diag.run()
    diags.append(diag)

#(RHmono,RVmono) = (3.31,1.88)
#lambda: 4.05
#lambda: 4.05 +/- 0.04
#energy: 4.98 +/- 0.10
#(RHanalyzer,RVanalyzer) = (4.17,0.77)
#'Monochromator', 'counter', 'before_sample_slit', 'sqw', 'after_sample_slit', 'analyzer', 'slit', 'detector_all']

In [ ]:
for diag in diags:
    diag.clear_views()
    diag.add_view("x", bins=50, limits=[-0.04,0.04])
    diag.add_view("y", bins=50, same_scale=False, limits=[-0.07,0.07])
    #diag.add_view("x","y",bins=[30,30])
    #diag.add_view("dx","dy")
    diag.add_view("dx")
    diag.add_view("dy")
    diag.plot()

In [ ]:
RVdiags = []


print(RVs)
for RV in RVs:
    print(RV)
    mycalc.parameters["RVmono"]=RV
    diag = ms.Diagnostics(mycalc)
    diag.settings(ncount=1e6, suppress_output=True,force_compile=False)
    diag.clear_points()
    diag.add_point(after="before_sample_slit")
#    diag.add_point(before="sqw")
    diag.run()
    RVdiags.append(diag)

In [ ]:
for diag in RVdiags:
    diag.clear_views()
    diag.add_view("x", bins=50, limits=[-0.04,0.04])
    diag.add_view("y", bins=50, same_scale=False, limits=[-0.07,0.07])
    #diag.add_view("x","y",bins=[30,30])
    #diag.add_view("dx","dy")
    diag.add_view("dx")
    diag.add_view("dy")
    diag.plot()

In [17]:
myinstrument.force_compile(False)
myinstrument.sim_neutrons(1e5)
counter=[]
detector=[]
mycalc = myinstrument.calculators["OriginCalc"]
for RV in RVs:
    for RH in RHs:
        mycalc.parameters["RHmono"].value = RH
        mycalc.parameters["RVmono"].value = RV
        #print(mycalc.parameters)
        myinstrument.run()
        counter.append(myinstrument.output["OriginCalc_data"].get_data()["data"])

---- Found 1 places in McStas output with keyword 'error'. 

         (negative time, miss next components, rounding errors, Nan, Inf).

Save [OriginCalc]
Detector: counter_I=0 counter_ERR=0 counter_N=0 "counter.dat"
Detector: detector_all_I=56.6516 detector_all_ERR=16.2704 detector_all_N=16 "detector_all.dat"

Finally [OriginCalc: /tmp/ThALES/OriginCalc_100]. Time: 0 [s] 
Isotropic_Sqw: sqw: Scattering fraction=0.130077 of incoming intensity
               Absorption fraction           =0.698955
               Single   scattering intensity =31796 (coh=0 inc=31796)
----------------------------------------------------------------------

INFO: Using directory: "/tmp/ThALES/OriginCalc_100"
INFO: Using existing c-file: ./OriginCalc.c
INFO: Using existing binary: ./OriginCalc.out
INFO: ===
Component: H53_7_parabolic (Guide_tapering)
           Height at the guide exit (h2): 0.120000 
Component: H53_7_parabolic (Guide_tapering)
           Width at the guide exit (w2): 0.024856 
(RHmono,RVmon

---- Found 1 places in McStas output with keyword 'error'. 

         (negative time, miss next components, rounding errors, Nan, Inf).

Save [OriginCalc]
Detector: counter_I=0 counter_ERR=0 counter_N=0 "counter.dat"
Detector: detector_all_I=62.645 detector_all_ERR=16.8728 detector_all_N=15 "detector_all.dat"

Finally [OriginCalc: /tmp/ThALES/OriginCalc_104]. Time: 0 [s] 
Isotropic_Sqw: sqw: Scattering fraction=0.136873 of incoming intensity
               Absorption fraction           =0.69908
               Single   scattering intensity =40410.8 (coh=0 inc=40410.8)
----------------------------------------------------------------------

INFO: Using directory: "/tmp/ThALES/OriginCalc_104"
INFO: Using existing c-file: ./OriginCalc.c
INFO: Using existing binary: ./OriginCalc.out
INFO: ===
Component: H53_7_parabolic (Guide_tapering)
           Height at the guide exit (h2): 0.120000 
Component: H53_7_parabolic (Guide_tapering)
           Width at the guide exit (w2): 0.024856 
(RHmono,RVm

---- Found 1 places in McStas output with keyword 'error'. 

         (negative time, miss next components, rounding errors, Nan, Inf).

Save [OriginCalc]
Detector: counter_I=24342.6 counter_ERR=19804 counter_N=14 "counter.dat"
Detector: detector_all_I=73.0231 detector_all_ERR=16.9905 detector_all_N=29 "detector_all.dat"

Finally [OriginCalc: /tmp/ThALES/OriginCalc_108]. Time: 1 [s] 
Isotropic_Sqw: sqw: Scattering fraction=0.134537 of incoming intensity
               Absorption fraction           =0.69893
               Single   scattering intensity =70015.6 (coh=0 inc=70015.6)
----------------------------------------------------------------------

INFO: Using directory: "/tmp/ThALES/OriginCalc_108"
INFO: Using existing c-file: ./OriginCalc.c
INFO: Using existing binary: ./OriginCalc.out
INFO: ===
Component: H53_7_parabolic (Guide_tapering)
           Height at the guide exit (h2): 0.120000 
Component: H53_7_parabolic (Guide_tapering)
           Width at the guide exit (w2): 0.024856 

---- Found 1 places in McStas output with keyword 'error'. 

         (negative time, miss next components, rounding errors, Nan, Inf).

Save [OriginCalc]
Detector: counter_I=17711 counter_ERR=20450.8 counter_N=3 "counter.dat"
Detector: detector_all_I=64.2767 detector_all_ERR=16.4674 detector_all_N=17 "detector_all.dat"

Finally [OriginCalc: /tmp/ThALES/OriginCalc_112]. Time: 0 [s] 
Isotropic_Sqw: sqw: Scattering fraction=0.127791 of incoming intensity
               Absorption fraction           =0.698976
               Single   scattering intensity =63814.2 (coh=0 inc=63814.2)
----------------------------------------------------------------------

INFO: Using directory: "/tmp/ThALES/OriginCalc_112"
INFO: Using existing c-file: ./OriginCalc.c
INFO: Using existing binary: ./OriginCalc.out
INFO: ===
Component: H53_7_parabolic (Guide_tapering)
           Height at the guide exit (h2): 0.120000 
Component: H53_7_parabolic (Guide_tapering)
           Width at the guide exit (w2): 0.024856 

---- Found 1 places in McStas output with keyword 'error'. 

         (negative time, miss next components, rounding errors, Nan, Inf).

Save [OriginCalc]
Detector: counter_I=178970 counter_ERR=57238.8 counter_N=36 "counter.dat"
Detector: detector_all_I=97.265 detector_all_ERR=21.167 detector_all_N=25 "detector_all.dat"

Finally [OriginCalc: /tmp/ThALES/OriginCalc_116]. Time: 0 [s] 
Isotropic_Sqw: sqw: Scattering fraction=0.131882 of incoming intensity
               Absorption fraction           =0.698965
               Single   scattering intensity =92606.6 (coh=0 inc=92606.6)
----------------------------------------------------------------------

INFO: Using directory: "/tmp/ThALES/OriginCalc_116"
INFO: Using existing c-file: ./OriginCalc.c
INFO: Using existing binary: ./OriginCalc.out
INFO: ===
Component: H53_7_parabolic (Guide_tapering)
           Height at the guide exit (h2): 0.120000 
Component: H53_7_parabolic (Guide_tapering)
           Width at the guide exit (w2): 0.024856 

...

---- Found 1 places in McStas output with keyword 'error'. 

         (negative time, miss next components, rounding errors, Nan, Inf).

Save [OriginCalc]
Detector: counter_I=327386 counter_ERR=83750.3 counter_N=42 "counter.dat"
Detector: detector_all_I=63.861 detector_all_ERR=17.4736 detector_all_N=15 "detector_all.dat"

Finally [OriginCalc: /tmp/ThALES/OriginCalc_128]. Time: 0 [s] 
Isotropic_Sqw: sqw: Scattering fraction=0.127823 of incoming intensity
               Absorption fraction           =0.698941
               Single   scattering intensity =73965.9 (coh=0 inc=73965.9)
----------------------------------------------------------------------

INFO: Using directory: "/tmp/ThALES/OriginCalc_128"
INFO: Using existing c-file: ./OriginCalc.c
INFO: Using existing binary: ./OriginCalc.out
INFO: ===
Component: H53_7_parabolic (Guide_tapering)
           Height at the guide exit (h2): 0.120000 
Component: H53_7_parabolic (Guide_tapering)
           Width at the guide exit (w2): 0.024856

---- Found 1 places in McStas output with keyword 'error'. 

         (negative time, miss next components, rounding errors, Nan, Inf).

Save [OriginCalc]
Detector: counter_I=77214.9 counter_ERR=39354 counter_N=19 "counter.dat"
Detector: detector_all_I=54.0334 detector_all_ERR=15.8644 detector_all_N=15 "detector_all.dat"

Finally [OriginCalc: /tmp/ThALES/OriginCalc_132]. Time: 0 [s] 
Isotropic_Sqw: sqw: Scattering fraction=0.128778 of incoming intensity
               Absorption fraction           =0.698941
               Single   scattering intensity =71342 (coh=0 inc=71342)
----------------------------------------------------------------------

INFO: Using directory: "/tmp/ThALES/OriginCalc_132"
INFO: Using existing c-file: ./OriginCalc.c
INFO: Using existing binary: ./OriginCalc.out
INFO: ===
Component: H53_7_parabolic (Guide_tapering)
           Height at the guide exit (h2): 0.120000 
Component: H53_7_parabolic (Guide_tapering)
           Width at the guide exit (w2): 0.024856 
(R

---- Found 1 places in McStas output with keyword 'error'. 

         (negative time, miss next components, rounding errors, Nan, Inf).

Save [OriginCalc]
Detector: counter_I=0.0267887 counter_ERR=0.0267887 counter_N=1 "counter.dat"
Detector: detector_all_I=58.8253 detector_all_ERR=16.3593 detector_all_N=15 "detector_all.dat"

Finally [OriginCalc: /tmp/ThALES/OriginCalc_136]. Time: 1 [s] 
Isotropic_Sqw: sqw: Scattering fraction=0.121488 of incoming intensity
               Absorption fraction           =0.698814
               Single   scattering intensity =39517.9 (coh=0 inc=39517.9)
----------------------------------------------------------------------

INFO: Using directory: "/tmp/ThALES/OriginCalc_136"
INFO: Using existing c-file: ./OriginCalc.c
INFO: Using existing binary: ./OriginCalc.out
INFO: ===
Component: H53_7_parabolic (Guide_tapering)
           Height at the guide exit (h2): 0.120000 
Component: H53_7_parabolic (Guide_tapering)
           Width at the guide exit (w2): 0.0

---- Found 1 places in McStas output with keyword 'error'. 

         (negative time, miss next components, rounding errors, Nan, Inf).

Save [OriginCalc]
Detector: counter_I=412393 counter_ERR=95955.9 counter_N=46 "counter.dat"
Detector: detector_all_I=80.356 detector_all_ERR=19.3487 detector_all_N=19 "detector_all.dat"

Finally [OriginCalc: /tmp/ThALES/OriginCalc_140]. Time: 0 [s] 
Isotropic_Sqw: sqw: Scattering fraction=0.129303 of incoming intensity
               Absorption fraction           =0.698907
               Single   scattering intensity =60740.4 (coh=0 inc=60740.4)
----------------------------------------------------------------------

INFO: Using directory: "/tmp/ThALES/OriginCalc_140"
INFO: Using existing c-file: ./OriginCalc.c
INFO: Using existing binary: ./OriginCalc.out
INFO: ===
Component: H53_7_parabolic (Guide_tapering)
           Height at the guide exit (h2): 0.120000 
Component: H53_7_parabolic (Guide_tapering)
           Width at the guide exit (w2): 0.024856

---- Found 1 places in McStas output with keyword 'error'. 

         (negative time, miss next components, rounding errors, Nan, Inf).

Save [OriginCalc]
Detector: counter_I=119985 counter_ERR=48758.8 counter_N=26 "counter.dat"
Detector: detector_all_I=97.4561 detector_all_ERR=21.2008 detector_all_N=28 "detector_all.dat"

Finally [OriginCalc: /tmp/ThALES/OriginCalc_144]. Time: 0 [s] 
Isotropic_Sqw: sqw: Scattering fraction=0.131457 of incoming intensity
               Absorption fraction           =0.698946
               Single   scattering intensity =102730 (coh=0 inc=102730)
----------------------------------------------------------------------

INFO: Using directory: "/tmp/ThALES/OriginCalc_144"
INFO: Using existing c-file: ./OriginCalc.c
INFO: Using existing binary: ./OriginCalc.out
INFO: ===
Component: H53_7_parabolic (Guide_tapering)
           Height at the guide exit (h2): 0.120000 
Component: H53_7_parabolic (Guide_tapering)
           Width at the guide exit (w2): 0.024856 

In [31]:
i=0
i_det=1
for RV in RVs:
    for RH in RHs:
        c = counter[i]
        det=c[1]
        print(i, "\t" , RV, RH, "\t\t" , det.Intensity, det.Error, det.Ncount)
        i=i+1

0 	 1.0 2.6 		 56.6515657 16.2704 16.0
1 	 1.0 2.9 		 50.45504574 15.1367 18.0
2 	 1.0 3.1999999999999997 		 53.25108236 15.4464 20.0
3 	 1.0 3.4999999999999996 		 63.9799955 17.0629 21.0
4 	 1.0 3.7999999999999994 		 62.64499308 16.8728 15.0
5 	 1.0 3.31 		 63.74411883 17.1537 22.0
6 	 1.3 2.6 		 42.51686025 13.7053 11.0
7 	 1.3 2.9 		 55.21798233 15.5668 19.0
8 	 1.3 3.1999999999999997 		 73.02312847 16.9905 29.0
9 	 1.3 3.4999999999999996 		 77.58587622 18.2759 28.0
10 	 1.3 3.7999999999999994 		 83.2727501 18.5729 28.0
11 	 1.3 3.31 		 85.78647256 18.9138 32.0
12 	 1.6 2.6 		 64.27672738 16.4674 17.0
13 	 1.6 2.9 		 88.20815373 19.4554 24.0
14 	 1.6 3.1999999999999997 		 98.60567426 20.8716 30.0
15 	 1.6 3.4999999999999996 		 94.82458681 20.5555 27.0
16 	 1.6 3.7999999999999994 		 97.26497809 21.167 25.0
17 	 1.6 3.31 		 104.551125 21.7258 31.0
18 	 1.9000000000000001 2.6 		 132.0188664 24.3551 33.0
19 	 1.9000000000000001 2.9 		 120.5604364 22.9575 33.0
20 	 1.9000000000000001 3.1

In [ ]:
mycalc.output

In [ ]:
print(myinstrument.output)

In [49]:
RHs = list(np.arange(1.5,3,0.2))
RVs = list(np.arange(0.3,1.2,0.2))
RVs.append(0.51)
RHs.append(1.41)
print(RHs)
print(RVs)

myinstrument.force_compile(False)
myinstrument.sim_neutrons(1e5)
counter=[]

mycalc = myinstrument.calculators["OriginCalc"]
for RV in RVs:
    for RH in RHs:
        mycalc.parameters["RHanalyzer"].value = RH
        mycalc.parameters["RVanalyzer"].value = RV
        #print(mycalc.parameters)
        myinstrument.run()
        counter.append(myinstrument.output["OriginCalc_data"].get_data()["data"])

[1.5, 1.7, 1.9, 2.0999999999999996, 2.3, 2.5, 2.6999999999999997, 2.8999999999999995, 1.41]
[0.3, 0.5, 0.7, 0.9000000000000001, 1.1, 0.51]
---- Found 1 places in McStas output with keyword 'error'. 

         (negative time, miss next components, rounding errors, Nan, Inf).

Save [OriginCalc]
Detector: counter_I=183734 counter_ERR=57131.8 counter_N=38 "counter.dat"
Detector: detector_all_I=95.8856 detector_all_ERR=21.3294 detector_all_N=26 "detector_all.dat"

Finally [OriginCalc: /tmp/ThALES/OriginCalc_292]. Time: 1 [s] 
Isotropic_Sqw: sqw: Scattering fraction=0.130101 of incoming intensity
               Absorption fraction           =0.698972
               Single   scattering intensity =103040 (coh=0 inc=103040)
----------------------------------------------------------------------

INFO: Using directory: "/tmp/ThALES/OriginCalc_292"
INFO: Using existing c-file: ./OriginCalc.c
INFO: Using existing binary: ./OriginCalc.out
INFO: ===
Component: H53_7_parabolic (Guide_tapering)
       

---- Found 1 places in McStas output with keyword 'error'. 

         (negative time, miss next components, rounding errors, Nan, Inf).

Save [OriginCalc]
Detector: counter_I=183716 counter_ERR=57141.2 counter_N=37 "counter.dat"
Detector: detector_all_I=108.075 detector_all_ERR=22.74 detector_all_N=33 "detector_all.dat"

Finally [OriginCalc: /tmp/ThALES/OriginCalc_296]. Time: 0 [s] 
Isotropic_Sqw: sqw: Scattering fraction=0.130229 of incoming intensity
               Absorption fraction           =0.698966
               Single   scattering intensity =102694 (coh=0 inc=102694)
----------------------------------------------------------------------

INFO: Using directory: "/tmp/ThALES/OriginCalc_296"
INFO: Using existing c-file: ./OriginCalc.c
INFO: Using existing binary: ./OriginCalc.out
INFO: ===
Component: H53_7_parabolic (Guide_tapering)
           Height at the guide exit (h2): 0.120000 
Component: H53_7_parabolic (Guide_tapering)
           Width at the guide exit (w2): 0.024856 
(

---- Found 1 places in McStas output with keyword 'error'. 

         (negative time, miss next components, rounding errors, Nan, Inf).

Save [OriginCalc]
Detector: counter_I=209796 counter_ERR=62874.2 counter_N=39 "counter.dat"
Detector: detector_all_I=96.7688 detector_all_ERR=21.2766 detector_all_N=27 "detector_all.dat"

Finally [OriginCalc: /tmp/ThALES/OriginCalc_300]. Time: 0 [s] 
Isotropic_Sqw: sqw: Scattering fraction=0.130029 of incoming intensity
               Absorption fraction           =0.698969
               Single   scattering intensity =103555 (coh=0 inc=103555)
----------------------------------------------------------------------

INFO: Using directory: "/tmp/ThALES/OriginCalc_300"
INFO: Using existing c-file: ./OriginCalc.c
INFO: Using existing binary: ./OriginCalc.out
INFO: ===
Component: H53_7_parabolic (Guide_tapering)
           Height at the guide exit (h2): 0.120000 
Component: H53_7_parabolic (Guide_tapering)
           Width at the guide exit (w2): 0.024856 

---- Found 1 places in McStas output with keyword 'error'. 

         (negative time, miss next components, rounding errors, Nan, Inf).

Save [OriginCalc]
Detector: counter_I=183716 counter_ERR=57141.2 counter_N=37 "counter.dat"
Detector: detector_all_I=163.602 detector_all_ERR=27.7251 detector_all_N=47 "detector_all.dat"

Finally [OriginCalc: /tmp/ThALES/OriginCalc_304]. Time: 1 [s] 
Isotropic_Sqw: sqw: Scattering fraction=0.130297 of incoming intensity
               Absorption fraction           =0.698971
               Single   scattering intensity =102678 (coh=0 inc=102678)
----------------------------------------------------------------------

INFO: Using directory: "/tmp/ThALES/OriginCalc_304"
INFO: Using existing c-file: ./OriginCalc.c
INFO: Using existing binary: ./OriginCalc.out
INFO: ===
Component: H53_7_parabolic (Guide_tapering)
           Height at the guide exit (h2): 0.120000 
Component: H53_7_parabolic (Guide_tapering)
           Width at the guide exit (w2): 0.024856 

---- Found 1 places in McStas output with keyword 'error'. 

         (negative time, miss next components, rounding errors, Nan, Inf).

Save [OriginCalc]
Detector: counter_I=183734 counter_ERR=57150.8 counter_N=36 "counter.dat"
Detector: detector_all_I=163.089 detector_all_ERR=28.1084 detector_all_N=41 "detector_all.dat"

Finally [OriginCalc: /tmp/ThALES/OriginCalc_308]. Time: 0 [s] 
Isotropic_Sqw: sqw: Scattering fraction=0.130639 of incoming intensity
               Absorption fraction           =0.69898
               Single   scattering intensity =103405 (coh=0 inc=103405)
----------------------------------------------------------------------

INFO: Using directory: "/tmp/ThALES/OriginCalc_308"
INFO: Using existing c-file: ./OriginCalc.c
INFO: Using existing binary: ./OriginCalc.out
INFO: ===
Component: H53_7_parabolic (Guide_tapering)
           Height at the guide exit (h2): 0.120000 
Component: H53_7_parabolic (Guide_tapering)
           Width at the guide exit (w2): 0.024856 


...

---- Found 1 places in McStas output with keyword 'error'. 

         (negative time, miss next components, rounding errors, Nan, Inf).

Save [OriginCalc]
Detector: counter_I=183716 counter_ERR=57141.2 counter_N=37 "counter.dat"
Detector: detector_all_I=100.918 detector_all_ERR=22.1106 detector_all_N=27 "detector_all.dat"

Finally [OriginCalc: /tmp/ThALES/OriginCalc_328]. Time: 0 [s] 
Isotropic_Sqw: sqw: Scattering fraction=0.130059 of incoming intensity
               Absorption fraction           =0.69897
               Single   scattering intensity =103114 (coh=0 inc=103114)
----------------------------------------------------------------------

INFO: Using directory: "/tmp/ThALES/OriginCalc_328"
INFO: Using existing c-file: ./OriginCalc.c
INFO: Using existing binary: ./OriginCalc.out
INFO: ===
Component: H53_7_parabolic (Guide_tapering)
           Height at the guide exit (h2): 0.120000 
Component: H53_7_parabolic (Guide_tapering)
           Width at the guide exit (w2): 0.024856 


---- Found 1 places in McStas output with keyword 'error'. 

         (negative time, miss next components, rounding errors, Nan, Inf).

Save [OriginCalc]
Detector: counter_I=183716 counter_ERR=57141.2 counter_N=37 "counter.dat"
Detector: detector_all_I=117.792 detector_all_ERR=24.1199 detector_all_N=32 "detector_all.dat"

Finally [OriginCalc: /tmp/ThALES/OriginCalc_332]. Time: 0 [s] 
Isotropic_Sqw: sqw: Scattering fraction=0.130372 of incoming intensity
               Absorption fraction           =0.698968
               Single   scattering intensity =102861 (coh=0 inc=102861)
----------------------------------------------------------------------

INFO: Using directory: "/tmp/ThALES/OriginCalc_332"
INFO: Using existing c-file: ./OriginCalc.c
INFO: Using existing binary: ./OriginCalc.out
INFO: ===
Component: H53_7_parabolic (Guide_tapering)
           Height at the guide exit (h2): 0.120000 
Component: H53_7_parabolic (Guide_tapering)
           Width at the guide exit (w2): 0.024856 

---- Found 1 places in McStas output with keyword 'error'. 

         (negative time, miss next components, rounding errors, Nan, Inf).

Save [OriginCalc]
Detector: counter_I=183716 counter_ERR=57141.2 counter_N=37 "counter.dat"
Detector: detector_all_I=93.7618 detector_all_ERR=21.4284 detector_all_N=25 "detector_all.dat"

Finally [OriginCalc: /tmp/ThALES/OriginCalc_336]. Time: 0 [s] 
Isotropic_Sqw: sqw: Scattering fraction=0.13026 of incoming intensity
               Absorption fraction           =0.698973
               Single   scattering intensity =102758 (coh=0 inc=102758)
----------------------------------------------------------------------

INFO: Using directory: "/tmp/ThALES/OriginCalc_336"
INFO: Using existing c-file: ./OriginCalc.c
INFO: Using existing binary: ./OriginCalc.out
INFO: ===
Component: H53_7_parabolic (Guide_tapering)
           Height at the guide exit (h2): 0.120000 
Component: H53_7_parabolic (Guide_tapering)
           Width at the guide exit (w2): 0.024856 


---- Found 1 places in McStas output with keyword 'error'. 

         (negative time, miss next components, rounding errors, Nan, Inf).

Save [OriginCalc]
Detector: counter_I=183716 counter_ERR=57141.2 counter_N=37 "counter.dat"
Detector: detector_all_I=163.602 detector_all_ERR=27.725 detector_all_N=47 "detector_all.dat"

Finally [OriginCalc: /tmp/ThALES/OriginCalc_340]. Time: 0 [s] 
Isotropic_Sqw: sqw: Scattering fraction=0.130297 of incoming intensity
               Absorption fraction           =0.698971
               Single   scattering intensity =102678 (coh=0 inc=102678)
----------------------------------------------------------------------

INFO: Using directory: "/tmp/ThALES/OriginCalc_340"
INFO: Using existing c-file: ./OriginCalc.c
INFO: Using existing binary: ./OriginCalc.out
INFO: ===
Component: H53_7_parabolic (Guide_tapering)
           Height at the guide exit (h2): 0.120000 
Component: H53_7_parabolic (Guide_tapering)
           Width at the guide exit (w2): 0.024856 


---- Found 1 places in McStas output with keyword 'error'. 

         (negative time, miss next components, rounding errors, Nan, Inf).

Save [OriginCalc]
Detector: counter_I=183734 counter_ERR=57150.8 counter_N=36 "counter.dat"
Detector: detector_all_I=163.089 detector_all_ERR=28.1084 detector_all_N=41 "detector_all.dat"

Finally [OriginCalc: /tmp/ThALES/OriginCalc_344]. Time: 0 [s] 
Isotropic_Sqw: sqw: Scattering fraction=0.130639 of incoming intensity
               Absorption fraction           =0.69898
               Single   scattering intensity =103405 (coh=0 inc=103405)
----------------------------------------------------------------------

INFO: Using directory: "/tmp/ThALES/OriginCalc_344"
INFO: Using existing c-file: ./OriginCalc.c
INFO: Using existing binary: ./OriginCalc.out
INFO: ===
Component: H53_7_parabolic (Guide_tapering)
           Height at the guide exit (h2): 0.120000 
Component: H53_7_parabolic (Guide_tapering)
           Width at the guide exit (w2): 0.024856 


In [50]:
i=0
i_det=1
intensities = []
counts = []
for RV in RVs:
    for RH in RHs:
        c = counter[i]
        det=c[1]
        #print(i, "\t" , RV, RH, "\t\t" , det.Intensity, det.Error, det.Ncount)
        intensities.append(det.Intensity)
        counts.append(det.Ncount)
        i=i+1

In [37]:
max(intensities)

array(151.3892)

In [38]:
max(counts)

array(43.)

In [39]:
intensities.index(max(intensities))

16

In [40]:
counts.index(max(counts))

16

In [64]:
i=0
for RV in RVs:
    for RH in RHs:
        print(RV, RH, counts[i])
        if i==counts.index(max(counts)):
            print("---> ",RV, RH)
        i=i+1

0.3 1.5 26.0
0.3 1.7 26.0
0.3 1.9 29.0
0.3 2.0999999999999996 32.0
0.3 2.3 33.0
0.3 2.5 32.0
0.3 2.6999999999999997 32.0
0.3 2.8999999999999995 30.0
0.3 1.41 27.0
0.5 1.5 40.0
0.5 1.7 42.0
0.5 1.9 45.0
0.5 2.0999999999999996 47.0
0.5 2.3 48.0
--->  0.5 2.3
0.5 2.5 41.0
0.5 2.6999999999999997 41.0
0.5 2.8999999999999995 41.0
0.5 1.41 36.0
0.7 1.5 36.0
0.7 1.7 39.0
0.7 1.9 41.0
0.7 2.0999999999999996 41.0
0.7 2.3 42.0
0.7 2.5 35.0
0.7 2.6999999999999997 36.0
0.7 2.8999999999999995 35.0
0.7 1.41 34.0
0.9000000000000001 1.5 32.0
0.9000000000000001 1.7 34.0
0.9000000000000001 1.9 36.0
0.9000000000000001 2.0999999999999996 35.0
0.9000000000000001 2.3 36.0
0.9000000000000001 2.5 30.0
0.9000000000000001 2.6999999999999997 31.0
0.9000000000000001 2.8999999999999995 31.0
0.9000000000000001 1.41 29.0
1.1 1.5 27.0
1.1 1.7 29.0
1.1 1.9 31.0
1.1 2.0999999999999996 31.0
1.1 2.3 32.0
1.1 2.5 26.0
1.1 2.6999999999999997 27.0
1.1 2.8999999999999995 27.0
1.1 1.41 25.0
0.51 1.5 40.0
0.51 1.7 42.0
0.51 1.9

In [52]:
RVs

[0.3, 0.5, 0.7, 0.9000000000000001, 1.1, 0.51]

In [53]:
RHs

[1.5,
 1.7,
 1.9,
 2.0999999999999996,
 2.3,
 2.5,
 2.6999999999999997,
 2.8999999999999995,
 1.41]

In [67]:
dist_sample_ana = 1.260
dist_ana_det = 0.640
import math
ana_focus = 1.0 / (1.0 / dist_sample_ana + 1.0 / dist_ana_det)

In [69]:

def rh(ana_focus):
    return 2*ana_focus/math.sin(math.pi/180*myinstrument.master["a6"].value/2)
def rv(ana_focus):
    return 2*ana_focus*math.sin(math.pi/180*myinstrument.master["a6"].value/2)


In [70]:
print(rv(ana_focus),rh(ana_focus))

0.5127168019934745 1.4053234004934378


In [71]:
ana_focus=dist_ana_det
print(rv(ana_focus),rh(ana_focus))

0.7731443839584138 2.119138461061533
